In [1]:
import random
from datetime import date, timedelta
from datetime import datetime as dt
import pandas as pd

In [2]:
def getRandomTime(min_x, max_x, start_time):
    start_time = dt.strptime(start_time, "%Y-%m-%dT%H:%M:%S:%f")
    # min and max duration of the activity %H:%M:%S
    min_duration = dt.combine(date.min, dt.strptime(min_x, "%H:%M:%S:%f").time())
    max_duration = dt.combine(date.min, dt.strptime(max_x, "%H:%M:%S:%f").time()) 
    # get a ramdom millisec value between min and max duration
    millisec = random.randint(0, int(((max_duration - min_duration).total_seconds())*1000))
    # sum the start date and the calculated random value
    min_x = dt.strptime(min_x, '%H:%M:%S:%f')
    b = timedelta(hours= min_x.hour, minutes = min_x.minute, seconds = min_x.second, milliseconds = min_x.microsecond)
    random_duration = start_time + timedelta(milliseconds = millisec) + b
    return (random_duration.strftime('%Y-%m-%dT%H:%M:%S:%f'), millisec+int(b.total_seconds()*1000))    

In [3]:
def addDelay(time):
    #delay tra le attivazioni, tra i 40 e i 200 secondi
    delay = random.randint(40, 200)
    return (dt.strptime(time, '%Y-%m-%dT%H:%M:%S:%f') + timedelta(seconds=delay)).strftime('%Y-%m-%dT%H:%M:%S:%f')

In [4]:
def createEventLog(rep, delay, skip):
    start_time = dt(2022, 1, 1, 11, 00, 0, 0).strftime('%Y-%m-%dT%H:%M:%S:%f')
    day = []
    for j in range(rep):
        if(delay == True):
        # if delay = 1 the lunch is delayed, 2 the dinner is delayed
            delay_flag = random.randint(1, 2)
        else: 
            delay_flag = 0
        
        if(skip == True):
        # if skip = 0 both skipped, skip = 1 the lunch is skipped, 2 the dinner is skipped
            skip_flag = random.randint(0, 2)
        else: 
            skip_flag = 3

        if(delay_flag == 1):
            # the lunch starts between 14 to 15, that is 3 to 4 hours from the start time
            eating = getRandomTime("03:00:00:00", "04:00:00:00", start_time)
            #calculate eating duration, between 30 and 60 minutes
            eating_complete = getRandomTime("00:30:00:00", "01:00:00:00", eating[0])
            day.append([j, "Eat", eating[0], eating_complete[0]])
            # day.append([j, "Eat", eating_complete[0], "complete"])
        # else, normal execution
        elif (skip_flag == 0 or skip_flag == 1): 
            eating_complete = getRandomTime("00:30:00:00", "01:00:00:00", start_time)
        else:
            #the process starts by eating, then it has leave and enter, then eating again, concluding with relax and sleep
            #get a random time to start eating, between 11:25 and 12:30
            eating = getRandomTime("00:25:00:00", "01:30:00:00", start_time)
            #calculate eating duration, between 30 and 60 minutes
            eating_complete = getRandomTime("00:30:00:00", "01:00:00:00", eating[0])
            day.append([j, "Eat", eating[0], eating_complete[0]])

        leave_start = addDelay(eating_complete[0])
        leave_complete = addDelay(leave_start)
        day.append([j, "Leave", leave_start, leave_complete])

        # around 7 hours after the start of the process, the person enters again to have dinner
        # the interval is between 7 and 8:30 hours, that is 18 to 19:30 
        enter_start = getRandomTime("07:00:00:00", "08:30:00:00", start_time)
        enter_complete = addDelay(enter_start[0])
        day.append([j, "Enter", enter_start[0], enter_complete])

        if(delay_flag == 2):  
            # the dinner is delayed, so it starts between 21:00 and 22:30
            # that is, after 10 and 11:30 hours wrt start_time
            eating2_start = getRandomTime("10:00:00:00", "11:30:00:00", start_time)
            # calculate eating duration, between 30 and 60 minutes
            eating2_complete = getRandomTime("00:30:00:00", "01:00:00:00", eating2_start[0])
            day.append([j, "Eat", eating2_start[0], eating2_complete[0]])
        elif (skip_flag == 0 or skip_flag == 2):
            eating2_complete = getRandomTime("00:30:00:00", "01:00:00:00", enter_complete)
        else:
            eating2_start = addDelay(enter_complete)
            # calculate eating duration, between 30 and 60 minutes
            eating2_complete = getRandomTime("00:30:00:00", "01:00:00:00", eating2_start)
            day.append([j, "Eat", eating2_start, eating2_complete[0]])

        relax_start = addDelay(eating2_complete[0])
        relax_complete = getRandomTime("01:00:00:00", "02:00:00:00", relax_start)
        day.append([j, "Relax", relax_start, relax_complete[0]])

        start_time_temp = dt.strptime(start_time, "%Y-%m-%dT%H:%M:%S:%f")
        start_time = (start_time_temp + timedelta(days=1)).strftime('%Y-%m-%dT%H:%M:%S:%f')   
      
    return day

In [5]:
from pm4py.objects.log.exporter.xes import exporter as xes_exporter
from pm4py.objects.conversion.log import converter as log_converter
import pm4py
import os

PATH = '../Scenario2'
if not os.path.exists(PATH):
    os.makedirs(PATH)

cols = ['case_id','activity', 'start_timestamp', 'complete_time']

#createEventLog(n of traces, delay, skip)
log = createEventLog(1000, False, False)
df = pd.DataFrame(log, columns = cols)
event_log = pm4py.format_dataframe(df, case_id='case_id', activity_key='activity', timestamp_key='complete_time', start_timestamp_key='start_timestamp', timest_format="%Y-%m-%dT%H:%M:%S:%f")
pm4py.write_xes(event_log, '../Scenario2/logNormal.xes')

log2 = createEventLog(1000, True, False)
df2 = pd.DataFrame(log2, columns = cols)
event_log2 = pm4py.format_dataframe(df2, case_id='case_id', activity_key='activity', timestamp_key='complete_time', start_timestamp_key='start_timestamp', timest_format="%Y-%m-%dT%H:%M:%S:%f")
pm4py.write_xes(event_log2, '../Scenario2/logDelay.xes')

log3 = createEventLog(1000, False, True)
df3 = pd.DataFrame(log3, columns = cols)
event_log3 = pm4py.format_dataframe(df3, case_id='case_id', activity_key='activity', timestamp_key='complete_time', start_timestamp_key='start_timestamp', timest_format="%Y-%m-%dT%H:%M:%S:%f")
pm4py.write_xes(event_log3, '../Scenario2/logAbsence.xes')

yes


c:\Users\gdfe\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
exporting log, completed traces :: 100%|██████████| 1000/1000 [00:00<00:00, 2161.42it/s]
